In [1]:
!pip install PyPDF2 sentence-transformers faiss-cpu



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install PyPDF2 sentence-transformers faiss-cpu fastapi uvicorn pydantic

In [4]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from openai import OpenAI
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import uvicorn
from typing import List, Optional
import os

app = FastAPI(title="RAG Chatbot API", description="Motherhood and Childcare Assistant")

# Global variables for RAG components
chunks = None
embedder = None
index = None
embeddings = None
client = None

class ChatRequest(BaseModel):
    message: str
    max_tokens: Optional[int] = 512

class ChatResponse(BaseModel):
    response: str
    retrieved_context: str

# ---- RAG Functions ----
def load_pdf_chunks(pdf_path, chunk_size=500):
    reader = PdfReader(pdf_path)
    full_text = ""
    for page in reader.pages:
        full_text += page.extract_text() + "\n"
    
    chunks = [full_text[i:i+chunk_size] for i in range(0, len(full_text), chunk_size)]
    return chunks

def embed_chunks(chunks, embedder):
    embeddings = embedder.encode(chunks)
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(np.array(embeddings))
    return index, embeddings

def retrieve_context(query, chunks, embedder, index, embeddings, top_k=3):
    query_embedding = embedder.encode([query])
    _, indices = index.search(np.array(query_embedding), top_k)
    return "\n---\n".join([chunks[i] for i in indices[0]])

@app.on_event("startup")
async def startup_event():
    """Initialize RAG components on startup"""
    global chunks, embedder, index, embeddings, client
    
    # Load PDF and setup RAG
    pdf_path = "./FAQ_for_Chatbot.pdf"
    if not os.path.exists(pdf_path):
        raise Exception(f"PDF file not found: {pdf_path}")
    
    print("Loading PDF and initializing RAG...")
    chunks = load_pdf_chunks(pdf_path)
    embedder = SentenceTransformer("intfloat/multilingual-e5-large-instruct")
    index, embeddings = embed_chunks(chunks, embedder)
    
    # Initialize OpenAI client
    client = OpenAI(
        api_key="sk-titCpkply6rFBcc6326yqTJzL20JeJJ9ACekZEij4nNCpClA",
        base_url="https://api.opentyphoon.ai/v1"
    )
    
    print("RAG Chatbot API initialized successfully!")

@app.post("/chat", response_model=ChatResponse)
async def chat(request: ChatRequest):
    """Chat endpoint for the RAG chatbot"""
    try:
        # Retrieve relevant context
        retrieved_context = retrieve_context(
            request.message, chunks, embedder, index, embeddings
        )
        
        # Prepare system prompt
        system_prompt = f"""
You are an AI assistant specialized in motherhood and childcare.
Use the following additional context to help answer the question:
{retrieved_context}
"""
        
        # Create conversation
        conversation = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": request.message}
        ]
        
        # Get response from OpenAI
        response = client.chat.completions.create(
            model="typhoon-v2-70b-instruct",
            messages=conversation,
            max_tokens=request.max_tokens
        )
        
        return ChatResponse(
            response=response.choices[0].message.content,
            retrieved_context=retrieved_context
        )
        
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/health")
async def health_check():
    """Health check endpoint"""
    return {"status": "healthy", "message": "RAG Chatbot API is running"}

@app.get("/")
async def root():
    """Root endpoint with API information"""
    return {
        "message": "Welcome to RAG Chatbot API",
        "endpoints": {
            "chat": "/chat - POST request with message",
            "health": "/health - Health check",
            "docs": "/docs - API documentation"
        }
    }

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

ไม่ผิดปกติค่ะ ในนมแม่มีโปรตีนที่ย่อยง่าย และในน้ำนมแม่ส่วนหน้าประกอบด้วยน้ำตาลแลคโตสและโอลิโกแซคคาไรด์ ซึ่งช่วยส่งเสริมจุลินทรีย์ชนิดดีในลำไส้ รวมถึงฮอร์โมนพรอสตาแกลนดินที่กระตุ้นการเคลื่อนตัวของลำไส้ ดังนั้น ทารกที่กินนมแม่มักถ่ายบ่อยและอุจจาระเหลวได้ หากอุจจาระของทารกไม่มีมูกเลือด ไม่มีกลิ่นเหม็นเน่า ร่วมกับดูดนมได้ดี ไม่มีไข้ ไม่มีซึม หรือร้องกวนผิดปกติ คุณแม่ก็ไม่ต้องกังวลค่ะ


In [ ]:
# ---- Run the API Server ----
# Uncomment the line below to start the server
# uvicorn.run(app, host="0.0.0.0", port=8000)

print("FastAPI application created!")
print("To start the server, run in terminal: uvicorn rag_api:app --reload --port 8000")
print("Then visit: http://localhost:8000/docs for API documentation")

In [ ]:
# ---- Test Client Example ----
import requests
import json

def test_chatbot_api(message="ลูกกินนมแม่แล้วถ่ายบ่อยผิดปกติหรือไม่?", port=8000):
    url = f"http://localhost:{port}/chat"
    
    data = {
        "message": message,
        "max_tokens": 512
    }
    
    try:
        response = requests.post(url, json=data)
        if response.status_code == 200:
            result = response.json()
            print("Bot Response:")
            print(result["response"])
            print("\n--- Retrieved Context ---")
            print(result["retrieved_context"])
        else:
            print(f"Error: {response.status_code}")
            print(response.text)
    except Exception as e:
        print(f"Connection error: {e}")
        print("Make sure the API server is running!")

# Uncomment to test (server must be running first)
# test_chatbot_api()

## How to Use the RAG Chatbot API

### 1. Start the API Server
Run this command in your terminal:
```bash
uvicorn rag_api:app --reload --port 8000
```

### 2. Test the API
- Visit `http://localhost:8000/docs` for interactive API documentation
- Use the test client function above
- Or make direct HTTP requests to `http://localhost:8000/chat`

### 3. API Endpoints
- `POST /chat` - Send a message to the chatbot
- `GET /health` - Check API health status
- `GET /` - API information
- `GET /docs` - Interactive API documentation

### 4. Request Format
```json
{
  "message": "Your question here",
  "max_tokens": 512
}
```

### 5. Response Format
```json
{
  "response": "AI assistant's answer",
  "retrieved_context": "Relevant context from PDF"
}
```